In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from urllib import parse
import requests
import datetime
from haversine import haversine

In [56]:
#전국 시군구 데이터 불러오기
title = pd.read_csv('전국_시군구.csv', encoding = 'utf-8-sig')
title

,주소
0,서울특별시 종로구
1,서울특별시 중구
2,서울특별시 용산구
3,서울특별시 성동구
4,서울특별시 광진구
...,...
224,경상남도 함양군
225,경상남도 거창군
226,경상남도 합천군
227,제주특별자치도 제주시


In [54]:
keyword = title.iloc[0:229]
keyword

,주소
0,서울특별시 종로구
1,서울특별시 중구
2,서울특별시 용산구
3,서울특별시 성동구
4,서울특별시 광진구
...,...
224,경상남도 함양군
225,경상남도 거창군
226,경상남도 합천군
227,제주특별자치도 제주시


In [4]:
#위도, 경도 코드 확인
import requests

location = "서울특별시 종로구"
url = f"https://dapi.kakao.com/v2/local/search/address.json?query={location}"
kakao_key = "47deaab96ca6bef51ce4cd14d197cf40"
result = requests.get(url, headers={"Authorization":f"KakaoAK {kakao_key}"})
json_obj = result.json()
# print(json_obj)

x = json_obj['documents'][0]['x']
y = json_obj['documents'][0]['y']
print(x , y)

126.978989954189 37.5735042429813


In [11]:
#전국 시군구 데이터에 위도, 경도 입히기
import requests

for key in range(len(keyword)): 
    try:
        location = keyword.iloc[key,0]
        url = f"https://dapi.kakao.com/v2/local/search/address.json?query={location}"
        kakao_key = "47deaab96ca6bef51ce4cd14d197cf40"
        result = requests.get(url, headers={"Authorization":f"KakaoAK {kakao_key}"})
        json_obj = result.json()
        #print(json_obj)
        
        print(json_obj['documents'][0]['x'],json_obj['documents'][0]['y'])
    except:
        print()

126.978989954189 37.5735042429813
126.997555182293 37.5638077703601
126.990582345331 37.5324310391314
127.036930141185 37.563427205337
127.082385189457 37.5385583136667
127.039657001091 37.5745229817122
127.092820287004 37.6065432383919
127.016702905651 37.5893588153919
127.025538071854 37.6397513275882
127.04721049936 37.6686914100284
127.056430475216 37.6543617567057
126.928940981464 37.6028174370823
126.9368156604 37.579161863979
126.901955141101 37.5662141900954
126.866501409661 37.5169884752609
126.849533759514 37.5509646154244
126.88750531451 37.4954330863648
126.895456780023 37.4568411485785
126.896278443882 37.5263671784802
126.93931505634 37.5124820423519
126.951561853868 37.4783683761333
127.032689317952 37.4835888228545
127.047377408384 37.5173319258532
127.105922243305 37.5145909234015
127.123792501252 37.5301933196159
129.032373132261 35.1062269835896
129.024298440459 35.0979235209887
129.045305770374 35.129266800067
129.067888802321 35.0912398945905
129.053165443787 35.16

In [8]:
ad = json_obj['documents'][0]['address_name']
x = json_obj['documents'][0]['x']
y = json_obj['documents'][0]['y']

print(ad, x , y)

제주특별자치도 서귀포시 126.559563466911 33.2540646255037


In [13]:
#시군구 끼리의 거리 계산하기

#전체 시군구 파일 불러오기
aiai = pd.read_csv("전국_시군구.csv")

#각 위도 경도 지정
lat = aiai.x
long = aiai.y

#순서쌍으로 저장
lat_long = []
for i in aiai.index:
    lat_long.append((lat[i],long[i]))

#거리를 저장할 df
distance = []

#이중 for문을 사용해 각 발생지 거리 계산
for i in lat_long:
    for j in lat_long:
        distance.append(haversine(i,j))

#리스트를 분리하기 위한 함수 정의
def list_chunk(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

#잘 나왔는지 확인
len(distance) #45796 = 214 * 214

#리스트 분할
distance_chunked = list_chunk(distance,229)

#최종 결과를 담아줄 df
final = DataFrame(index = aiai.주소, columns = aiai.주소)

#인덱스를 사용한 지점별 값 집어넣기
for i in range(len(final)):
    final.iloc[:,i] = distance_chunked[i]   

#한글이 깨지지 않기 위한 encoding
final.to_csv("data/전국시군구거리.csv",mode = 'w',encoding='euc-kr')

In [57]:
#전국 시군구X전국 발생지 끼리의 거리 계산하기

#전국 발생지 파일 불러오기
aiai = pd.read_csv("전국_시군구_좌표.csv")
aiai2 = pd.read_csv("전국_발생지_좌표.csv")

aiai

,주소,x,y
0,서울특별시 종로구,126.978990,37.573504
1,서울특별시 중구,126.997555,37.563808
2,서울특별시 용산구,126.990582,37.532431
3,서울특별시 성동구,127.036930,37.563427
4,서울특별시 광진구,127.082385,37.538558
...,...,...,...
224,경상남도 함양군,127.725161,35.520527
225,경상남도 거창군,127.909503,35.686685
226,경상남도 합천군,128.165791,35.566670
227,제주특별자치도 제주시,126.531171,33.499534


In [58]:
aiai2

,주소,월,x,y,xy
0,강원도 인제군,1,128.170351,38.069710,"128.170351505777,38.069709741838"
1,강원도 횡성군,1,127.985080,37.491783,"127.985079956092,37.4917831601354"
2,경기도 광주시,1,127.255139,37.429394,"127.255138719721,37.4293941934866"
3,경기도 김포시,1,126.715658,37.615269,"126.715657922927,37.6152686898333"
4,경기도 남양주시,1,127.216480,37.636027,"127.21648030719,37.6360272346499"
...,...,...,...,...,...
102,전라남도 장성군,12,126.784919,35.301858,"126.784918877012,35.3018576771354"
103,전라남도 진도군,12,126.263446,34.486829,"126.263446186439,34.486828620348"
104,전라북도 남원시,12,127.390396,35.416363,"127.390396427422,35.4163627529983"
105,충청남도 예산군,12,126.848642,36.682623,"126.848642241312,36.6826228017861"


In [59]:
#각 위도 경도 지정
lat = aiai.x
long = aiai.y

lat2 = aiai2.x
long2 = aiai2.y

In [5]:
#순서쌍으로 저장
lat_long = []
for i in aiai.index:
    lat_long.append((lat[i],long[i]))

In [6]:
#순서쌍으로 저장
lat_long2 = []
for j in aiai2.index:
    lat_long2.append((lat2[j],long2[j]))

In [37]:
lat_long

[(126.97899, 37.57350424),
 (126.99755520000001, 37.56380777),
 (126.99058229999999, 37.53243104),
 (127.03693009999999, 37.56342721),
 (127.08238519999999, 37.53855831),
 (127.039657, 37.57452298),
 (127.0928203, 37.60654324),
 (127.01670290000001, 37.58935882),
 (127.0255381, 37.63975133),
 (127.04721049999999, 37.66869141),
 (127.05643049999999, 37.65436176),
 (126.928941, 37.60281744),
 (126.93681570000001, 37.57916186),
 (126.9019551, 37.56621419),
 (126.86650139999999, 37.51698848),
 (126.8495338, 37.55096462),
 (126.8875053, 37.49543309),
 (126.89545679999999, 37.45684115),
 (126.8962784, 37.52636718),
 (126.93931509999999, 37.51248204),
 (126.9515619, 37.47836838),
 (127.03268929999999, 37.48358882),
 (127.0473774, 37.51733193),
 (127.10592220000001, 37.51459092),
 (127.12379250000001, 37.53019332),
 (129.0323731, 35.10622698),
 (129.02429840000002, 35.09792352),
 (129.0453058, 35.129266799999996),
 (129.0678888, 35.09123989),
 (129.0531654, 35.16285545),
 (129.0938578, 35.1963

In [38]:
lat_long2

[(128.17035149999998, 38.06970974),
 (127.98508000000001, 37.49178316),
 (127.2551387, 37.42939419),
 (126.71565790000001, 37.61526869),
 (127.2164803, 37.63602723),
 (127.0287159, 37.26358468),
 (127.27979029999999, 37.0080705),
 (127.0457861, 37.7853137),
 (127.6370588, 37.29842337),
 (127.07535120000001, 38.09651865),
 (127.17748229999998, 37.24102998),
 (127.43508929999999, 37.27226759),
 (126.7798768, 37.76004138),
 (127.11268840000001, 36.99226067),
 (127.20033290000002, 37.89499283),
 (126.8314774, 37.1995372),
 (127.909503, 35.68668463),
 (128.32236129999998, 34.97306184),
 (128.74660749999998, 35.50381727),
 (129.03727279999998, 35.33499206),
 (128.10770630000002, 35.18022283),
 (127.75130800000001, 35.067305600000005),
 (128.1867719, 36.58652737),
 (128.2829346, 35.9190934),
 (129.3431646, 36.01899543),
 (128.9805706, 35.21219795),
 (127.2891953, 36.48006491),
 (126.7672135, 34.64206153),
 (127.2849687, 34.61108202),
 (127.46270359999998, 35.20249636),
 (126.7108293, 35.01589

In [39]:
#거리를 저장할 df
distance = []

#이중 for문을 사용해 각 발생지 거리 계산
for i in lat_long:
    for j in lat_long2:
        distance.append(haversine(i,j))

In [40]:
print(distance)

[136.6785509259385, 112.00880869261515, 32.19294372689124, 29.4133719658328, 26.73862631934324, 21.465653872382525, 50.58679026772945, 16.005777921202803, 75.48584634929689, 36.623912250271495, 31.367899170639102, 54.61100112144234, 25.399774240373873, 41.67886811191453, 32.71877900920094, 29.875901318822645, 164.2375063847945, 231.2994110717895, 242.02000200861596, 275.42284567103377, 205.04086963434685, 189.66657730875477, 150.05208843406913, 183.39913433937514, 283.7340920710057, 275.0003381278465, 81.09859774148768, 196.99000665239586, 201.72182357914318, 168.2920573402528, 173.11402318038353, 180.4491148604505, 187.96126471265833, 194.83276110498753, 204.28305944157663, 187.52990050376064, 193.41894657415676, 174.5552479960413, 204.1705801844887, 167.8253438226998, 145.81169473287926, 110.44976788805235, 118.74976140071671, 125.8466484507234, 135.72387714225522, 134.51646784023063, 58.71758064183932, 87.07103568757569, 78.7638654299901, 104.53409711700796, 52.48664570592909, 52.97

In [10]:
#리스트를 분리하기 위한 함수 정의
def list_chunk(lst, n):
    return [lst[i:i+n] for i in range(0, len(lst), n)]

#잘 나왔는지 확인
print(len(distance)) #45796 = 214 * 214

24503


In [41]:
#리스트 분할
distance_chunked = list_chunk(distance,107)

#최종 결과를 담아줄 df
final = DataFrame(index = aiai.주소, columns = aiai2.주소)

In [52]:
distance_chunked

[[136.6785509259385,
  112.00880869261515,
  32.19294372689124,
  29.4133719658328,
  26.73862631934324,
  21.465653872382525,
  50.58679026772945,
  16.005777921202803,
  75.48584634929689,
  36.623912250271495,
  31.367899170639102,
  54.61100112144234,
  25.399774240373873,
  41.67886811191453,
  32.71877900920094,
  29.875901318822645,
  164.2375063847945,
  231.2994110717895,
  242.02000200861596,
  275.42284567103377,
  205.04086963434685,
  189.66657730875477,
  150.05208843406913,
  183.39913433937514,
  283.7340920710057,
  275.0003381278465,
  81.09859774148768,
  196.99000665239586,
  201.72182357914318,
  168.2920573402528,
  173.11402318038353,
  180.4491148604505,
  187.96126471265833,
  194.83276110498753,
  204.28305944157663,
  187.52990050376064,
  193.41894657415676,
  174.5552479960413,
  204.1705801844887,
  167.8253438226998,
  145.81169473287926,
  110.44976788805235,
  118.74976140071671,
  125.8466484507234,
  135.72387714225522,
  134.51646784023063,
  58.7175

In [20]:
idx = list(final.index)

In [43]:
len(distance_chunked)

229

In [44]:
np.array(distance_chunked)

array([[136.67855093, 112.00880869,  32.19294373, ..., 152.00973548,
         61.23723651, 107.7919932 ],
       [134.84624824, 109.91602103,  30.02906341, ..., 150.80719832,
         61.14700262, 106.03538306],
       [136.14934513, 110.61745324,  30.21910659, ..., 149.00852374,
         58.91529001, 104.82575717],
       ...,
       [171.98930815, 133.52879006, 162.17820537, ...,  86.82579174,
        164.79447405,  83.17850826],
       [358.05745766, 313.56203457, 274.36249048, ..., 159.84088076,
        214.38289207, 220.75510403],
       [370.95514944, 326.36088139, 289.28925208, ..., 171.59375588,
        230.09272434, 233.43839301]])

In [45]:
final = pd.DataFrame(np.array(distance_chunked), index = aiai.주소, columns = aiai2.주소)

In [46]:
final.head()

주소,강원도 인제군,강원도 횡성군,경기도 광주시,경기도 김포시,경기도 남양주시,경기도 수원시,경기도 안성시,경기도 양주시,경기도 여주시,경기도 연천군,...,경기도 성남시,경기도 양평군,경상북도 구미시,경상북도 상주시,인천광역시 서구,전라남도 장성군,전라남도 진도군,전라북도 남원시,충청남도 예산군,충청북도 옥천군
주소,,,,,,,,,,,,,,,,,,,,,
서울특별시 종로구,136.678551,112.008809,32.192944,29.413372,26.738626,21.465654,50.586790,16.005778,75.485846,36.623912,...,19.364388,56.819140,181.109330,153.073889,33.739742,153.121528,219.634009,152.009735,61.237237,107.791993
서울특별시 중구,134.846248,109.916021,30.029063,31.532942,24.820816,20.393195,48.752570,15.770278,73.325269,36.712124,...,17.280186,54.703285,179.032175,150.975225,35.773165,152.818929,219.834005,150.807198,61.147003,106.035383
서울특별시 용산구,136.149345,110.617453,30.219107,31.065409,26.062266,18.487736,47.677504,18.008214,73.595032,38.934999,...,16.890980,55.331156,178.521787,150.550363,34.987572,150.618798,217.598586,149.008524,58.915290,104.825757
서울특별시 성동구,130.627643,105.541126,25.878973,35.890977,20.551079,20.101122,46.049250,14.895194,69.082990,35.974739,...,13.849163,50.341452,175.396922,147.252051,40.148423,153.600023,221.559832,149.636400,62.476294,103.470566
서울특별시 광진구,126.234782,100.425218,20.561640,41.098603,16.283954,19.366961,41.864250,17.030474,63.769725,37.414327,...,9.349334,45.165508,170.296559,142.100291,45.187195,153.057051,222.161717,146.835216,62.857176,99.283130


In [47]:
#한글이 깨지지 않기 위한 encoding
final.to_csv("전국시군구X발생시군구.csv",mode = 'w',encoding='euc-kr')

In [15]:
final

주소,서울특별시 종로구,서울특별시 중구,서울특별시 용산구,서울특별시 성동구,서울특별시 광진구,서울특별시 동대문구,서울특별시 중랑구,서울특별시 성북구,서울특별시 강북구,서울특별시 도봉구,...,경상남도 창녕군,경상남도 고성군,경상남도 남해군,경상남도 하동군,경상남도 산청군,경상남도 함양군,경상남도 거창군,경상남도 합천군,제주특별자치도 제주시,제주특별자치도 서귀포시
주소,,,,,,,,,,,,,,,,,,,,,
서울특별시 종로구,0.000000,136.678551,172.677921,15.732680,28.411402,94.579501,275.356425,173.392614,109.202337,203.042531,...,118.077971,336.330138,115.303328,134.239579,44.616004,90.077260,235.513112,155.043724,125.161968,118.199753
서울특별시 중구,2.163883,112.008809,183.647053,11.725277,161.283974,51.923240,276.896315,182.322973,135.529033,208.498874,...,90.752494,101.900970,157.417810,147.056051,111.179161,107.297266,180.100039,155.554033,125.474061,183.539584
서울특별시 용산구,3.034940,32.192944,110.261693,11.721902,228.356706,83.285804,265.914093,185.507742,150.281862,200.751759,...,70.137236,103.360762,116.079257,151.750538,81.518326,136.796071,195.335854,153.778287,171.171386,102.159087
서울특별시 성동구,6.477862,29.413372,135.887670,10.493958,239.353624,92.495514,283.067855,197.320714,119.709329,194.181982,...,81.796303,246.091611,197.862027,132.009955,49.222916,169.650136,212.957227,154.224028,110.471558,72.481747
서울특별시 광진구,11.732792,26.738626,82.645823,5.854636,272.803730,89.675164,268.460989,198.274074,112.818074,53.130240,...,84.476705,266.338462,179.084692,169.635776,95.544011,184.657019,223.607807,152.542817,168.322942,50.536984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
경상남도 함양군,161.442104,185.367595,10.966355,28.524439,83.958622,273.485626,264.449164,77.368246,287.033769,151.289755,...,326.561690,139.873699,109.591532,106.274566,172.815652,287.201188,150.281862,130.557095,105.095097,108.004756
경상남도 거창군,164.237506,52.594751,8.314121,52.431973,152.886093,273.113332,74.761373,63.278329,205.818285,116.402120,...,176.470033,155.808714,82.707126,132.009955,165.555885,183.837074,151.045458,108.040167,83.919925,106.859306
경상남도 합천군,189.537918,87.075336,6.745829,27.923096,80.104397,273.685958,196.626178,99.156162,207.846342,122.896867,...,278.746522,190.714953,113.533114,179.032366,128.211723,180.614809,151.886433,163.861869,103.461735,108.916937


In [50]:
#발생지와 시군구 거리 정렬(가까운순서대로 km앞에, 주소가 뒤에 나오게)

#각 지점별 거리가 저장된 df
final

#결과를 저장할 df 생성
near_order = DataFrame(index = final.index,columns = range(len(final)))

#column 하나를 기준으로 정렬, 반올림, 문자열로 변환, 
for i in range(len(near_order)):
    temp = DataFrame(final.iloc[i])
    temp = temp.sort_values(by=temp.columns[0])
    temp = temp.astype(str)
    temp = temp.reset_index()
    temp = temp.iloc[:,1] + ' ' + temp['주소']
    near_order.iloc[i,:] = temp

del near_order[0]

near_order.to_csv("전국시군구X발생시군구(가까운순서).csv",mode = 'w',encoding='euc-kr')

In [ ]:
#발생지끼리의 거리 가까운순서대로 (주소가 앞에, km가 뒤에나오게)

#각 지점별 거리가 저장된 df
final

#결과를 저장할 df 생성
near_order = DataFrame(index = final.index,columns = range(len(final)))

#column 하나를 기준으로 정렬, 반올림, 문자열로 변환, 
for i in range(len(near_order)):
    temp = DataFrame(final.iloc[:,i])
    temp = temp.sort_values(by=temp.columns[0])
    temp = temp.astype(str)
    temp = temp.reset_index()
    temp = temp['주소'] + ' ' +'('+ temp.iloc[:,1] + 'Km' + ')'
    near_order.iloc[i,:] = temp

del near_order[0]

near_order.to_csv("발생지간거리(가까운순서).csv",mode = 'w',encoding='euc-kr')